# STAGE B EXERCISE 

The dataset for the remainder of this quiz is the Appliances Energy Prediction data. The data set is at 10 min for about 4.5 months. The house temperature and humidity conditions were monitored with a ZigBee wireless sensor network. Each wireless node transmitted the temperature and humidity conditions around 3.3 min. Then, the wireless data was averaged for 10 minutes periods. The energy data was logged every 10 minutes with m-bus energy meters. Weather from the nearest airport weather station (Chievres Airport, Belgium) was downloaded from a public data set from Reliable Prognosis (rp5.ru), and merged together with the experimental data sets using the date and time column. Two random variables have been included in the data set for testing the regression models and to filter out non predictive attributes (parameters). The attribute information can be seen below.

Attribute Information:

Date, time year-month-day hour:minute:second

Appliances, energy use in Wh

lights, energy use of light fixtures in the house in Wh

T1, Temperature in kitchen area, in Celsius

RH_1, Humidity in kitchen area, in %

T2, Temperature in living room area, in Celsius

RH_2, Humidity in living room area, in %

T3, Temperature in laundry room area

RH_3, Humidity in laundry room area, in %

T4, Temperature in office room, in Celsius

RH_4, Humidity in office room, in %

T5, Temperature in bathroom, in Celsius

RH_5, Humidity in bathroom, in %

T6, Temperature outside the building (north side), in Celsius

RH_6, Humidity outside the building (north side), in %

T7, Temperature in ironing room , in Celsius

RH_7, Humidity in ironing room, in %

T8, Temperature in teenager room 2, in Celsius

RH_8, Humidity in teenager room 2, in %

T9, Temperature in parents room, in Celsius

RH_9, Humidity in parents room, in %

To, Temperature outside (from Chievres weather station), in Celsius

Pressure (from Chievres weather station), in mm Hg

RH_out, Humidity outside (from Chievres weather station), in %

Wind speed (from Chievres weather station), in m/s

Visibility (from Chievres weather station), in km

Tdewpoint (from Chievres weather station), Â°C

rv1, Random variable 1, nondimensional

rv2, Random variable 2, nondimensional

In [60]:
import random
random.seed(0)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
# import error metrics 
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

# import regression models
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso

In [50]:
# load the data
df = pd.read_csv('energydata_complete.csv')
df.head()

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,...,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,...,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,...,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,...,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,...,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


In [51]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19735 entries, 0 to 19734
Data columns (total 29 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   date         19735 non-null  object 
 1   Appliances   19735 non-null  int64  
 2   lights       19735 non-null  int64  
 3   T1           19735 non-null  float64
 4   RH_1         19735 non-null  float64
 5   T2           19735 non-null  float64
 6   RH_2         19735 non-null  float64
 7   T3           19735 non-null  float64
 8   RH_3         19735 non-null  float64
 9   T4           19735 non-null  float64
 10  RH_4         19735 non-null  float64
 11  T5           19735 non-null  float64
 12  RH_5         19735 non-null  float64
 13  T6           19735 non-null  float64
 14  RH_6         19735 non-null  float64
 15  T7           19735 non-null  float64
 16  RH_7         19735 non-null  float64
 17  T8           19735 non-null  float64
 18  RH_8         19735 non-null  float64
 19  T9  

In [52]:
data = df[["T2","T6"]]
data.head()

,T2,T6
0,19.2,7.026667
1,19.2,6.833333
2,19.2,6.560000
3,19.2,6.433333
4,19.2,6.366667


### From the dataset, fit a linear model on the relationship between the temperature in the living room in Celsius (x = T2) and the temperature outside the building (y = T6). What is the R^2 value in two d.p.?

In [76]:
# instantiite the linear regression object
linear_model = LinearRegression()
X = np.array(data["T2"]).reshape(-1, 1)
y = np.array(data["T6"]).reshape(-1, 1)


# split the daaset into training and test dataset
x_train, x_test, y_train, y_test = train_test_split(X,
                                                    y, 
                                                   test_size=0.3,
                                                   train_size=0.7,
                                                   random_state=42)
# instantiite the linear regression object
linear_model = LinearRegression()

#fit the model to the training dataset
linear_model.fit(x_train,y_train)

#obtain predictions
predicted_values = linear_model.predict(x_test)
r2_score = r2_score(y_test, predicted_values)
print( 'The RSquared value is ',round(r2_score,2))

The RSquared value is  0.64


### Normalize the dataset using the MinMaxScaler after removing the following columns: [“date”, “lights”]. The target variable is “Appliances”. 

### Use a 70-30 train-test set split with a random state of 42 (for reproducibility). 

### Run a multiple linear regression using the training set and evaluate your model on the test set. Answer the following questions:

* What is the Mean Absolute Error (in two decimal places)?
* What is the Residual Sum of Squares (in two decimal places)?
* What is the Root Mean Squared Error (in three decimal places)?
* What is the  Coefficient of  Determination (in two decimal places)?


In [53]:
# drop data and light column bcos they contain non- numeric table
df1 = df.drop(columns= ['date', 'lights'])

In [56]:
# standardized the dataset using MinMaxScaler
scaler = MinMaxScaler()
normalised_df = pd.DataFrame(scaler.fit_transform(df1), columns=df1.columns)
features_df = df1.drop(columns= ['Appliances']) #
Appliances_target = normalised_df["Appliances"]

# split the daaset into training and test dataset
x_train, x_test, y_train, y_test = train_test_split(features_df,
                                                    Appliances_target,
                                                   test_size=0.3,
                                                   train_size=0.7,
                                                   random_state=42)

# instantiite the linear regression object
linear_model = LinearRegression()

#fit the model to the training dataset
linear_model.fit(x_train, y_train)

#obtain predictions
predicted_values = linear_model.predict(x_test)

# obtain the mean absolute error
mae = mean_absolute_error(y_test, predicted_values)
print('The MAE value is ',round(mae,2))

# obtain the residual sum of square
rss = np.sum(np.square(y_test - predicted_values))
print( 'The RSS value is ',round(rss,2))

# obtain the residual mean square error
rmse = np.sqrt(mean_squared_error(y_test, predicted_values))
print('The RMSE value is ',round(rmse,3))

# obtain the residual square 
r2_score = r2_score(y_test, predicted_values)

print( 'The RSquared value is ',round(r2_score,2))

The MAE value is  0.05
The RSS value is  45.35
The RMSE value is  0.088
The RSquared value is  0.15


## Train a ridge regression model with an alpha value of 0.4. Is there any change to the root mean squared error (RMSE) when evaluated on the test set?

In [61]:
# instantiate ridge regrgession 
ridge_reg = Ridge(alpha =-0.4)
ridge_reg.fit(x_train, y_train)

# instantiate lasso regrgession 
lasso_reg =  Lasso(alpha=0.001)
lasso_reg.fit(x_train, y_train)

# create a function that will be t check the model weight
def get_weights_df(model, feat, col_name):
    # this function returns the weight of every feature
    weights = pd.Series(model.coef_, feat.columns).sort_values()
    weights_df = pd.DataFrame(weights).reset_index()
    weights_df.columns = ["Features", col_name]
    weights_df[col_name].round(3)
    return weights_df

linear_model_weights = get_weights_df(linear_model, x_train, 'Linear_Model_Weight')
ridge_weights_df = get_weights_df(ridge_reg, x_train, 'Ridge_Weight')
lasso_weights_df = get_weights_df(lasso_reg, x_train, 'Lasso_weight')

In [70]:
ridge_reg.fit(x_train, y_train)
predicted_values = ridge_reg.predict(x_test)

rmse = np.sqrt(mean_squared_error(y_test, predicted_values))
print('The RMSE value is ',round(rmse,3))

The RMSE value is  0.088


### Obtain the feature weights from your linear model above. Which features have the lowest and highest weights respectively?

In [62]:
linear_model_weights.sort_values(by = "Linear_Model_Weight")

,Features,Linear_Model_Weight
0,T9,-0.019765
1,T2,-0.017168
2,RH_2,-0.012842
3,T_out,-0.010349
4,RH_8,-0.005401
5,RH_9,-0.001647
6,RH_7,-0.001582
7,T5,-0.001496
8,RH_out,-0.001022
9,T1,-0.000346


## What is the new RMSE with the lasso regression? (Answer should be in three (3)decimal places)

In [69]:
lasso_reg.fit(x_train, y_train)
predicted_values = lasso_reg.predict(x_test)

rmse = np.sqrt(mean_squared_error(y_test, predicted_values))
print('The RMSE value is ',round(rmse,3))

The RMSE value is  0.088


In [64]:
lasso_weights_df

,Features,Lasso_weight
0,T9,-1.512603e-02
1,T2,-9.668148e-03
2,RH_2,-9.300608e-03
3,RH_8,-5.077541e-03
4,T_out,-3.325131e-03
5,RH_7,-1.644124e-03
6,T5,-1.476185e-03
7,RH_9,-1.431118e-03
8,RH_out,-1.934555e-05
9,T1,-0.000000e+00
